In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00


# Pipeline


In [1]:
import gradio as gr
from huggingface_hub import InferenceClient
from fastapi import FastAPI
from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel
from typing import List, Dict
import uvicorn
import threading

# Função para exibir o Swagger UI
def swagger_ui():
    return get_swagger_ui_html(
        openapi_url="/openapi.json",
        title="Swagger UI",
    )

# Inicializa a API FastAPI
app = FastAPI(title="Sugestão de Colunas API", version="1.0", description="API para sugerir colunas com base em embeddings.")

# Inicializa o cliente de inferência do Hugging Face Hub
client = InferenceClient()

# Lista de colunas esperadas
expected_columns = ["data", "valor", "categoria", "produto", "preço", "quantidade"]

# Modelo de entrada da API
class ColumnRequest(BaseModel):
    columns: List[str]

# Modelo de saída da API
class ColumnResponse(BaseModel):
    suggestions: Dict[str, Dict[str, float]]

@app.post("/suggest-columns", response_model=ColumnResponse, summary="Sugere colunas com base em embeddings")
async def suggest_columns(request: ColumnRequest):
    suggestions = {}

    # Obtém os embeddings das colunas esperadas usando a Inference API
    expected_embeddings = client.encode(expected_columns, model="Supabase/gte-small")

    for col in request.columns:
        # Obtém o embedding da coluna atual usando a Inference API
        col_embedding = client.encode(col, model="Supabase/gte-small")

        # Calcula a similaridade de cosseno
        similarities = [cosine_similarity(col_embedding, emb) for emb in expected_embeddings]

        # Encontra a coluna esperada com maior similaridade
        best_match_idx = similarities.index(max(similarities))
        best_match = expected_columns[best_match_idx]
        score = similarities[best_match_idx]

        suggestions[col] = {"match": best_match, "confidence": round(score, 2)}

    return {"suggestions": suggestions}

# Função para calcular a similaridade de cosseno
def cosine_similarity(a, b):
    dot_product = sum(x * y for x, y in zip(a, b))
    magnitude_a = sum(x**2 for x in a)**0.5
    magnitude_b = sum(x**2 for x in b)**0.5
    return dot_product / (magnitude_a * magnitude_b)

# Crie uma rota para o Swagger UI
app.mount("/static", StaticFiles(directory="static"), name="static")
app.get("/docs", response_class=HTMLResponse)(swagger_ui)

# Interface do Gradio
iface = gr.Interface(
    fn=suggest_columns,
    inputs=gr.JSON(label="Colunas"),
    outputs=gr.JSON(label="Sugestões"),
    title="Sugestão de Colunas",
    description="API para sugerir colunas com base em embeddings.",
)

# Inicie a API FastAPI e o Gradio
if __name__ == "__main__":
  config = uvicorn.Config(app, host="0.0.0.0", port=8000)
  server = uvicorn.Server(config)

  # Inicie o Gradio em uma thread separada
  threading.Thread(target=iface.launch, kwargs={"share": True}).start()

  # Inicie o servidor FastAPI
  server.run()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [5]:
!pip install gradio fastapi uvicorn python-multipart huggingface_hub sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
